In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import autograd.numpy as np
from autograd import grad

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

print("Imported all!")

Imported all!


In [2]:
file_path = 'techSalaries2017.csv'

df = pd.read_csv(file_path)
print("Size before dropping: ", df.size)
print(df.isnull().sum())
for i, column_name in enumerate(df.columns.values.tolist()):
    print("index:", str(i) + ";", "spec num:", str(i+1) + ";", "df['{}']".format(column_name))

Size before dropping:  1691334
company                        5
title                          0
location                       0
totalyearlycompensation        0
basesalary                     0
stockgrantvalue                0
bonus                          0
yearsofexperience              0
yearsatcompany                 0
gender                     19540
Masters_Degree                 0
Bachelors_Degree               0
Doctorate_Degree               0
Highschool                     0
Some_College                   0
Race_Asian                     0
Race_White                     0
Race_Two_Or_More               0
Race_Black                     0
Race_Hispanic                  0
Race                       40215
Education                  32272
Age                            0
Height                         0
Zodiac                         0
SAT                            0
GPA                            0
dtype: int64
index: 0; spec num: 1; df['company']
index: 1; spec num: 2; df['t

In [3]:
# drop columns we are not using for the predictions
columns_to_drop = ['company', 'title', 'location']

for column in columns_to_drop:
    df = df.drop(column, axis=1)
    print("\nRemoving '{}' column from the prediction data".format(column))


# create a dataset for the total_yearly_compensation_prediction by removing `basesalary`, `stockgrantvalue`, `bonus` columns
new_columns_to_drop = ['basesalary', 'stockgrantvalue', 'bonus']
for column in new_columns_to_drop:
    df = df.drop(column, axis=1)
    print("\nRemoving '{}' column from the dataset for total_yearly_compensation prediction".format(column))

df.head()


Removing 'company' column from the prediction data

Removing 'title' column from the prediction data

Removing 'location' column from the prediction data

Removing 'basesalary' column from the dataset for total_yearly_compensation prediction

Removing 'stockgrantvalue' column from the dataset for total_yearly_compensation prediction

Removing 'bonus' column from the dataset for total_yearly_compensation prediction


,totalyearlycompensation,yearsofexperience,yearsatcompany,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,...,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,Age,Height,Zodiac,SAT,GPA
0,127000,1.5,1.5,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,38,68.590,5,829,2.13
1,100000,5.0,3.0,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,34,68.059,7,993,2.77
2,310000,8.0,0.0,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,42,61.343,3,1200,3.26
3,372000,7.0,5.0,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,28,66.495,3,1170,3.07
4,157000,5.0,3.0,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,34,69.015,2,1115,2.91


In [4]:
def return_shape(df):
    return "Rows: " + str(df.shape[0]) + "; Columns: " + str(df.shape[1])

print(return_shape(df))

Rows: 62642; Columns: 21


In [5]:
# let's drop all the columns that contain incomplete information about person's race or education
new_df = df.dropna()
print(return_shape(new_df))

Rows: 21591; Columns: 21


In [6]:
# let's leave one dummy variable out for the education 
df = new_df.drop("Doctorate_Degree", axis=1)

# let's leave one dummy variable out for the race 
df = df.drop("Race_Two_Or_More", axis=1)
print(return_shape(df))

Rows: 21591; Columns: 19


### QUESTION 1: Using multiple linear regression: What is the best predictor of total annual compensation, how much variance is explained by this predictor vs. the full multiple regression model?

In [8]:
import matplotlib.pyplot as plt

def scatter_plot(ax, predictor, outcome, title):
    ax.scatter(predictor, outcome, s=7, alpha=0.1, color="orange")
    ax.set_title(f"{title} vs annual compensation")
    ax.set_xlabel(title)
    ax.set_ylabel("total annual compensation")


def plot_scatter(data, header, outcome_variable):
    indices = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0)]
    fig, axs = plt.subplots(3, 2, figsize=(12, 8))

    for i, name in zip(indices, header):
        scatter_plot(axs[i[0], i[1]], data[name], outcome_variable, name)

def normalize_dataset(dataset):
    mean = dataset.mean()
    standard_deviation = dataset.std()

    normalized_data = (dataset - mean) / standard_deviation
    return pd.DataFrame(normalized_data)

normalized_gpa_col = normalize_dataset(df['GPA'])
normalized_sat_col = normalize_dataset(df['SAT'])

normalized_gpa_and_sat = pd.DataFrame(normalized_gpa_col, normalized_sat_col, columns=['GPA_normalized', 'SAT_normalized'])

# We want to plot these values to identify what are the most useful predictors
predictors_list = ['GPA', 'SAT', 'Age', 'yearsofexperience', 'yearsatcompany']
outcome_variable = df['totalyearlycompensation']
my_predictors_df = pd.concat([df[predictors_list], normalized_gpa_and_sat], axis=1)

my_predictors_df.head()

plot_scatter(df[predictors_list], predictors_list, outcome_variable)
plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()




ValueError: Index data must be 1-dimensional